In [7]:
class Triangle:
    a: int
    b: int
    c: int
    n_dots = 3


    def __init__(self, a_, b_, c_):
        self.a = a_
        self.b = b_
        self.c = c_

tr_1 = Triangle(3, 4, 5)
tr_2 = Triangle(1, 100, 200)

In [12]:
class Triangle:
    a: int
    b: int
    c: int
    n_dots = 3

    def __init__(self, a_, b_, c_):
        if a_ + b_ <= c_ or a_ + c_ <= b_ or b_ + c_ <= a_:
            raise ValueError('triangle inequality does not hold')
        
        self.a = a_
        self.b = b_
        self.c = c_

    def area(self):
        p = 0.5 * (self.a + self.b + self.c)
        area = (p * (p - self.a) * (p - self.b) * (p - self.c)) ** 0.5
        return area  

tr_1 = Triangle(3, 4, 5)
tr_2 = Triangle(10, 12, 15)
square_1 = tr_1.area()
square_2 = tr_2.area()

In [ ]:
class Rectangle(Triangle):
    n_dots = 4

    def __init__(self, a_, b_):
        self.a = a_
        self.b = b_
        
    def area(self):
        return self.a * self.b

In [ ]:
class BaseFigure:
    n_dots = None

    def __init__(self):
        self.validate()

    def validate(self):
        raise NotImplementedError()

    def area(self):
        raise NotImplementedError()


In [ ]:
class Triangle(BaseFigure):
    n_dots = 3

    def __init__(self, a_, b_, c_):
        self.a = a_
        self.b = b_
        self.c = c_
        self.validate()
        super().__init__()

    def validate(self):
        if self.a + self.b <= self.c or self.a + self.c <= self.b or self.b + self.c <= self.a:
            raise ValueError('triangle inequality does not hold')
        return self.a, self.b, self.c

    def area(self):
        p = 0.5 * (self.a + self.b + self.c)
        area = (p * (p - self.a) * (p - self.b) * (p - self.c)) ** 0.5
        return area
    
class Rectangle(BaseFigure):
    n_dots = 4

    def __init__(self, a_, b_):
        self.a = a_
        self.b = b_
        self.validate()
        super().__init__()

    def validate(self):
        return self.a, self.b

    def area(self):
        return self.a * self.b

In [ ]:
class Circle(BaseFigure):
    n_dots = float('inf')

    def __init__(self, r_):
        self.r = r_
        self.validate()
        super().__init__()

    def validate(self):
        return self.r

    def area(self):
        return 3.14 * self.r ** 2

In [6]:
class Vector():
    def __init__(self, coords):
        self.coords = coords

    def __add__(self, other):
        if len(self.coords) != len(other.coords):
            raise ValueError(f'left and right lengths differ: {len(self.coords)} != {len(other.coords)}')
        return Vector([x + y for x, y in zip(self.coords, other.coords)])
    
    def __str__(self):
        return f'[{", ".join(map(str, self.coords))}]'
    
    def __mul__(self, other):
        if isinstance(other, Vector):
            if len(self.coords) != len(other.coords):
                raise ValueError(f'left and right lengths differ: {len(self.coords)} != {len(other.coords)}')
            return sum(x * y for x, y in zip(self.coords, other.coords))
        return Vector([x * other for x in self.coords])
    
    def __eq__(self, other):
        return self.coords == other.coords
    
    def __abs__(self):
        return sum(x ** 2 for x in self.coords) ** 0.5

[5, 7, 9]


In [3]:
class ParsesCookies:
    def __init__(self, request):
        self.request = request

    def cookies(self):
        return self.request.get('cookies', {})
    
    def is_authed(self):
        return 'auth_key' in self.cookies()
    

class ParsesBody:
    def __init__(self, request):
        self.request = request

    def body(self):
        return self.request.get('body', '')
    

class ParsesHeaders:
    def __init__(self, request):
        self.request = request

    def headers(self):
        return self.request.get('headers', {})
    
    def need_json(self):
        return self.headers().get('content-type') == 'application/json'
    

class Handler(ParsesCookies, ParsesBody, ParsesHeaders):
    def __init__(self, request):
        ParsesCookies.__init__(self, request)
        ParsesBody.__init__(self, request)
        ParsesHeaders.__init__(self, request)

In [4]:
import json

class JsonHandler(ParsesBody, ParsesHeaders):
    def __init__(self, request):
        self.request = request

    def process(self):
        
        if self.need_json():
            try:
                body_json = json.loads(self.body())
                return len(body_json.keys())
            except:
                return None
        else:
            return None

In [7]:
# Пример использования
r = {'body': '{"a": 123, "b": 1234}',
     'headers': {'content-type': 'application/json'}
    }
print(JsonHandler(r).process())

2


In [8]:
class SecureTextHandler(ParsesBody, ParsesCookies):
    def __init__(self, request):
        self.request = request

    def process(self):
        if not self.is_authed():
            return None
        body = self.body()
        return len(body)

In [9]:
# Примеры
r = {'cookies': {'auth_key': '123'},
     'body': 'hello'
    }
print(SecureTextHandler(r).process())
# 5

r = {'cookies': {},
     'body': 'hello'
    }
print(SecureTextHandler(r).process())
# None

5
None
